In [2]:
import pandas as pd
import Generator as qm
import HelperUtils as helper
import time

import json
from pandas.io.json import json_normalize

import importlib
importlib.reload(qm)
importlib.reload(helper)

<module 'HelperUtils' from '/Users/hagenaadfock/Documents/GitHub/UU/thesis/tkge-nlp/TKGQuestionMaster/HelperUtils.py'>

In [103]:
pd.read_json('generated_data/0602/YAGO11k_questions_v1_0602.json')

,yes_no_qe,yes_no_an,when_qe,when_an,until_when_qe,until_when_an,left_open_qe,left_open_an,right_open_qe,right_open_an,when_to_when_qe,when_to_when_an,duration_qe,duration_an,object_has_years,subject_has_years
0,Was Beverly Adams born in Edmonton in 1945?,yes,When was Beverly Adams born in Edmonton?,[1945],0,0,0,0,0,0,0,0,0,0,False,False
1,Was Rae Dawn Chong born in Edmonton in 1961?,yes,When was Rae Dawn Chong born in Edmonton?,[1961],0,0,0,0,0,0,0,0,0,0,False,False
2,Was Daniel Fernandes (footballer) born in Edmo...,yes,When was Daniel Fernandes (footballer) born in...,[1983],0,0,0,0,0,0,0,0,0,0,False,False
3,Was Mike Comrie born in Edmonton in 1980?,yes,When was Mike Comrie born in Edmonton?,[1980],0,0,0,0,0,0,0,0,0,0,False,False
4,Was Jordan Peterson born in Edmonton in 1962?,yes,When was Jordan Peterson born in Edmonton?,[1962],0,0,0,0,0,0,0,0,0,0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16752,Was Benjamin Butler affiliated to Greenback Pa...,yes,When was Benjamin Butler affiliated to Greenba...,"[1874, 1875, 1876, 1877, 1878, 1879, 1880, 188...",Until when was Benjamin Butler affiliated to G...,1889,From 1874 until was Benjamin Butler affiliated...,1874,From 1874 until was Benjamin Butler affiliated...,1874,From when to when was Benjamin Butler affiliat...,"[1874, 1875, 1876, 1877, 1878, 1879, 1880, 188...",For how long was Benjamin Butler affiliated to...,15,False,False
16753,Was Charles E. Cunningham affiliated to Greenb...,yes,When was Charles E. Cunningham affiliated to G...,"[1876, 1877, 1878, 1879, 1880, 1881, 1882, 188...",Until when was Charles E. Cunningham affiliate...,1886,From 1876 until was Charles E. Cunningham affi...,1876,From 1876 until was Charles E. Cunningham affi...,1876,From when to when was Charles E. Cunningham af...,"[1876, 1877, 1878, 1879, 1880, 1881, 1882, 188...",For how long was Charles E. Cunningham affilia...,10,False,False
16754,Was John Quincy Adams affiliated to Anti-Mason...,yes,When was John Quincy Adams affiliated to Anti-...,"[1833, 1834, 1835, 1836, 1837, 1838]",Until when was John Quincy Adams affiliated to...,1838,From 1833 until was John Quincy Adams affiliat...,1833,From 1833 until was John Quincy Adams affiliat...,1833,From when to when was John Quincy Adams affili...,"[1833, 1834, 1835, 1836, 1837, 1838]",For how long was John Quincy Adams affiliated ...,5,False,False
16755,Was Thaddeus Stevens affiliated to Anti-Masoni...,yes,When was Thaddeus Stevens affiliated to Anti-M...,"[1828, 1829, 1830, 1831, 1832, 1833, 1834, 183...",Until when was Thaddeus Stevens affiliated to ...,1838,From 1828 until was Thaddeus Stevens affiliate...,1828,From 1828 until was Thaddeus Stevens affiliate...,1828,From when to when was Thaddeus Stevens affilia...,"[1828, 1829, 1830, 1831, 1832, 1833, 1834, 183...",For how long was Thaddeus Stevens affiliated t...,10,False,False


In [3]:
df = pd.read_csv('../data/processed_datasets/TKG_readable_YAGO11k.csv', index_col=0)
# removing days and months
df['from'] = df['from'].apply(lambda x: x[:x.find('-')])
df['until'] = df['until'].apply(lambda x: x[:x.find('-')])

predicates = {'is married to', 'has won prize', 'owns', 'graduated from', 'works at', 'plays for', 'created',
              'is affiliated to'}
imputable_set = {'has won prize', 'created', 'graduated from'}
not_imputable_set = {'is affiliated to', 'is married to', 'owns', 'plays for', 'works at'}


def is_part_of(predicate, set):
    if predicate in set:
        return True
    return False

missing_df = df[(df['until'] == '####')]
missing_df[missing_df['predicate'].isin(imputable_set)]

,subject,predicate,object,from,until,object_has_years,subject_has_years
7296,Al Gore,has won prize,Nobel Peace Prize,2007,####,False,False
7297,Linus Pauling,has won prize,Nobel Peace Prize,1962,####,False,False
7298,Andrei Sakharov,has won prize,Nobel Peace Prize,1975,####,False,False
7299,Martin Luther King Jr.,has won prize,Nobel Peace Prize,1964,####,False,False
7300,Joseph Rotblat,has won prize,Nobel Peace Prize,1995,####,False,False
...,...,...,...,...,...,...,...
19115,Antony Sher,created,I.D. (play),2003,####,False,False
19117,Gordon Tipple,created,The Master (Doctor Who),1996,####,False,False
19118,Lil Wayne,created,Throw That Boy Pussy,2015,####,False,False
19119,David Small,created,Stitches (book),2009,####,False,False


In [6]:
set(df.predicate.tolist())

{'created',
 'died in',
 'graduated from',
 'has won prize',
 'is affiliated to',
 'is married to',
 'owns',
 'plays for',
 'was born in',
 'works at'}

In [107]:
missing_df[missing_df['predicate'].isin(not_imputable_set)]

,subject,predicate,object,from,until,object_has_years,subject_has_years
3343,Vincent Wong (Hong Kong actor),is married to,Yoyo Chen,2011,####,False,False
3347,Wendy Moniz,is married to,Frank Grillo,2000,####,False,False
3351,Choi Won-young,is married to,Shim Yi-young,2014,####,False,False
3352,Athena Chu,is married to,Paul Wong (musician),2012,####,False,False
3353,Eric Cantona,is married to,Rachida Brakni,2007,####,False,False
...,...,...,...,...,...,...,...
20481,Sergio Mattarella,is affiliated to,Christian Democracy (Italy),1994,####,False,False
20482,Enrico Letta,is affiliated to,Christian Democracy (Italy),1994,####,False,False
20496,Katja Kipping,is affiliated to,The Left (Germany),2007,####,False,False
20497,Sahra Wagenknecht,is affiliated to,The Left (Germany),2007,####,False,False


In [108]:
df['until'] = df.apply(lambda x:
                       x['from'] if (x['until'] == '####' and is_part_of(x['predicate'], imputable_set)) else x[
                           'until'], axis=1)

df['not_imputable_until'] = df.apply(lambda x:
                                     True if (x['until'] == '####' and is_part_of(x['predicate'],
                                                                                  not_imputable_set)) else False,
                                     axis=1)
df = df[df['not_imputable_until'] == False]
df = df.reset_index().iloc[:, 1:]

In [109]:
unsuitable_row_ids = []
for i, entry in df.iterrows():
    if entry['from'].__contains__('#') or entry['from'].__contains__('-') or str(entry['until']).__contains__(
            '#') or str(entry['until']).__contains__('-') or entry['from'] == '':
        unsuitable_row_ids.append(i)
print(unsuitable_row_ids)

[2847, 14209, 14448, 14727, 14823, 15055, 15056, 15057, 15058, 15136, 15137, 15167, 15177, 15178, 15179, 15180, 15181, 15182, 15320, 15495, 15674, 15725, 15970]


In [110]:
df.iloc[unsuitable_row_ids, :]

,subject,predicate,object,from,until,object_has_years,subject_has_years,not_imputable_until
2847,Michael Houghton (virologist),was born in,United Kingdom,195#,195#,False,False,False
14209,Tylman van Gameren,created,"Brühl Palace, Warsaw",17##,18##,False,False,False
14448,Yue Fei,created,Xing Yi Quan,12##,19##,False,False,False
14727,Arthur Blomfield,created,"St Mary Magdalene, Richmond",20##,20##,False,False,False
14823,Karl Friedrich Schinkel,created,Ehrenburg Palace,19##,19##,False,False,False
15055,Euripides,created,The Bacchae,,,False,False,False
15056,Euripides,created,Medea (play),,,False,False,False
15057,Euripides,created,Hippolytus (play),,,False,False,False
15058,Euripides,created,The Trojan Women,,,False,False,False
15136,John Nash (architect),created,St David's Cathedral,18##,18##,False,False,False


In [111]:
# Dropping all entries where the dates are ambiguous
df = df.drop(unsuitable_row_ids)

In [112]:
lemma = False
time_indication = True

df_falsy_yes_no_questions = df.apply(lambda x: qm.formulate_yes_no_question(x['subject'], x['predicate'], x['object'],
                                                x['from'], x['until'], lemma=lemma,
                                                time_indication=time_indication,
                                                count_of_falsy_year_question=2), axis=1)

In [114]:
df_falsy_yes_no_questions.to_json('generated_data/0606/questions_v3_yes_no_robust_raw.json')

In [116]:
with open('generated_data/0606/questions_v3_yes_no_robust_raw.json') as json_data:
    data = json.load(json_data)

df_yes_no_robust = pd.DataFrame()
index = 0
for i, entry in data.items():
    for qe in entry:
        df_yes_no_robust.loc[index, 'qe_index'] = i
        df_yes_no_robust.loc[index, 'qe'] = qe[0]
        df_yes_no_robust.loc[index, 'an'] = qe[1]
        index += 1

df_yes_no_robust.to_json('generated_data/0606/questions_v3_yes_no_robust.json')

In [75]:
def convert_qm_output_to_two_df_columns(dataframe, prefix):
    dataframe[f'{prefix}_qe'] = dataframe[prefix].apply(lambda x: x[0][0] if x is not None else None)
    dataframe[f'{prefix}_an'] = dataframe[prefix].apply(lambda x: x[0][1] if x is not None else None)


def generate_questions(df, lemma, time_indication):
    start_time = time.time()

    questions = pd.DataFrame()
    questions['yes_no'] = df.apply(lambda x:
                                   qm.formulate_yes_no_question(x['subject'], x['predicate'], x['object'],
                                                                x['from'], x['until'], lemma=lemma,
                                                                time_indication=time_indication), axis=1)
    questions['when'] = df.apply(lambda x:
                                 qm.formulate_simple_when_question(x['subject'], x['predicate'], x['object'],
                                                                   x['from'], x['until'],
                                                                   time_indication=time_indication, lemma=lemma),
                                 axis=1)

    questions['when_to_when'] = df.apply(lambda x:
                                         qm.formulate_when_to_when_question(x['subject'], x['predicate'], x['object'],
                                                                            x['from'], x['until'],
                                                                            time_indication=time_indication,
                                                                            lemma=lemma),
                                         axis=1)

    questions['from_when'] = df.apply(lambda x:
                                      qm.formulate_from_or_until_question(x['subject'], x['predicate'], x['object'],
                                                                          x['from'], x['until'],
                                                                          is_from_question=True,
                                                                          time_indication=time_indication, lemma=lemma),
                                      axis=1)

    questions['until_when'] = df.apply(lambda x:
                                       qm.formulate_from_or_until_question(x['subject'], x['predicate'], x['object'],
                                                                           x['from'], x['until'],
                                                                           is_from_question=False,
                                                                           time_indication=time_indication,
                                                                           lemma=lemma),
                                       axis=1)

    questions['left_open'] = df.apply(lambda x:
                                      qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                         x['object'],
                                                                                         x['from'], x['until'],
                                                                                         right_open=False,
                                                                                         time_indication=time_indication,
                                                                                         lemma=lemma),
                                      axis=1)

    questions['right_open'] = df.apply(lambda x:
                                       qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                          x['object'],
                                                                                          x['from'], x['until'],
                                                                                          right_open=True,
                                                                                          time_indication=time_indication,
                                                                                          lemma=lemma),
                                       axis=1)
    questions['duration'] = df.apply(lambda x:
                                     qm.formulate_duration_question(x['subject'], x['predicate'],
                                                                    x['object'],
                                                                    x['from'], x['until'],
                                                                    time_indication=time_indication,
                                                                    lemma=lemma),
                                     axis=1)

    print("--- %s seconds ---" % (time.time() - start_time))

    convert_qm_output_to_two_df_columns(questions, 'yes_no')
    convert_qm_output_to_two_df_columns(questions, 'when')
    convert_qm_output_to_two_df_columns(questions, 'until_when')
    convert_qm_output_to_two_df_columns(questions, 'left_open')
    convert_qm_output_to_two_df_columns(questions, 'right_open')
    convert_qm_output_to_two_df_columns(questions, 'when_to_when')
    convert_qm_output_to_two_df_columns(questions, 'duration')

    return questions


In [91]:
def generate_questions_v2(df, lemma, time_indication):
    questions = pd.DataFrame()
    questions['left_open'] = df.apply(lambda x:
                                      qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                         x['object'],
                                                                                         x['from'], x['until'],
                                                                                         right_open=False,
                                                                                         time_indication=time_indication,
                                                                                         lemma=lemma),
                                      axis=1)

    questions['right_open'] = df.apply(lambda x:
                                       qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                          x['object'],
                                                                                          x['from'], x['until'],
                                                                                          right_open=True,
                                                                                          time_indication=time_indication,
                                                                                          lemma=lemma),
                                       axis=1)

    convert_qm_output_to_two_df_columns(questions, 'left_open')
    convert_qm_output_to_two_df_columns(questions, 'right_open')
    return questions

In [97]:
# V1 not lemmatized without time indication
v1_questions = generate_questions_v2(df, lemma=False, time_indication=False)
v2_questions = generate_questions_v2(df, lemma=True, time_indication=False)
v3_questions = generate_questions_v2(df, lemma=False, time_indication=True)
v4_questions = generate_questions_v2(df, lemma=True, time_indication=True)

In [99]:
questions = [v1_questions, v2_questions, v3_questions, v4_questions]
questions

[                                               left_open  \
 0                                                   None   
 1                                                   None   
 2                                                   None   
 3                                                   None   
 4                                                   None   
 ...                                                  ...   
 16752  [(From when until 1874 was Benjamin Butler aff...   
 16753  [(From when until 1876 was Charles E. Cunningh...   
 16754  [(From when until 1833 was John Quincy Adams a...   
 16755  [(From when until 1828 was Thaddeus Stevens af...   
 16756  [(From when until 1828 was Joseph Ritner affil...   
 
                                               right_open  \
 0                                                   None   
 1                                                   None   
 2                                                   None   
 3                    

In [84]:
# V1 not lemmatized without time indication
v1_questions = generate_questions(df, lemma=False, time_indication=False)
# V2 lemmatized without time indication
v2_questions = generate_questions(df, lemma=True, time_indication=False)
# V3 not lemmatized with time indication
v3_questions = generate_questions(df, lemma=False, time_indication=True)
# V4 lemmatized and time indication
v4_questions = generate_questions(df, lemma=True, time_indication=True)

questions = [v1_questions, v2_questions, v3_questions, v4_questions]

KeyboardInterrupt: 

In [78]:
v1_questions

,yes_no,when,when_to_when,from_when,until_when,left_open,right_open,duration,yes_no_qe,yes_no_an,...,until_when_qe,until_when_an,left_open_qe,left_open_an,right_open_qe,right_open_an,when_to_when_qe,when_to_when_an,duration_qe,duration_an
0,"[(Was Beverly Adams born in Edmonton in 1945?,...","[(When was Beverly Adams born in Edmonton?, [1...",None,None,None,None,None,None,Was Beverly Adams born in Edmonton in 1945?,yes,...,None,NaN,None,NaN,None,NaN,None,None,None,NaN
1,[(Was Rae Dawn Chong born in Edmonton in 1961?...,"[(When was Rae Dawn Chong born in Edmonton?, [...",None,None,None,None,None,None,Was Rae Dawn Chong born in Edmonton in 1961?,yes,...,None,NaN,None,NaN,None,NaN,None,None,None,NaN
2,[(Was Daniel Fernandes (footballer) born in Ed...,[(When was Daniel Fernandes (footballer) born ...,None,None,None,None,None,None,Was Daniel Fernandes (footballer) born in Edmo...,yes,...,None,NaN,None,NaN,None,NaN,None,None,None,NaN
3,"[(Was Mike Comrie born in Edmonton in 1980?, y...","[(When was Mike Comrie born in Edmonton?, [198...",None,None,None,None,None,None,Was Mike Comrie born in Edmonton in 1980?,yes,...,None,NaN,None,NaN,None,NaN,None,None,None,NaN
4,[(Was Jordan Peterson born in Edmonton in 1962...,"[(When was Jordan Peterson born in Edmonton?, ...",None,None,None,None,None,None,Was Jordan Peterson born in Edmonton in 1962?,yes,...,None,NaN,None,NaN,None,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16752,[(Was Benjamin Butler affiliated to Greenback ...,[(When was Benjamin Butler affiliated to Green...,[(From when to when was Benjamin Butler affili...,[(Since when was Benjamin Butler affiliated to...,[(Until when was Benjamin Butler affiliated to...,[(From 1874 until was Benjamin Butler affiliat...,[(From 1874 until was Benjamin Butler affiliat...,[(For how long was Benjamin Butler affiliated ...,Was Benjamin Butler affiliated to Greenback Pa...,yes,...,Until when was Benjamin Butler affiliated to G...,1889.0,From 1874 until was Benjamin Butler affiliated...,1874.0,From 1874 until was Benjamin Butler affiliated...,1889.0,From when to when was Benjamin Butler affiliat...,"[1874, 1875, 1876, 1877, 1878, 1879, 1880, 188...",For how long was Benjamin Butler affiliated to...,15.0
16753,[(Was Charles E. Cunningham affiliated to Gree...,[(When was Charles E. Cunningham affiliated to...,[(From when to when was Charles E. Cunningham ...,[(Since when was Charles E. Cunningham affilia...,[(Until when was Charles E. Cunningham affilia...,[(From 1876 until was Charles E. Cunningham af...,[(From 1876 until was Charles E. Cunningham af...,[(For how long was Charles E. Cunningham affil...,Was Charles E. Cunningham affiliated to Greenb...,yes,...,Until when was Charles E. Cunningham affiliate...,1886.0,From 1876 until was Charles E. Cunningham affi...,1876.0,From 1876 until was Charles E. Cunningham affi...,1886.0,From when to when was Charles E. Cunningham af...,"[1876, 1877, 1878, 1879, 1880, 1881, 1882, 188...",For how long was Charles E. Cunningham affilia...,10.0
16754,[(Was John Quincy Adams affiliated to Anti-Mas...,[(When was John Quincy Adams affiliated to Ant...,[(From when to when was John Quincy Adams affi...,[(Since when was John Quincy Adams affiliated ...,[(Until when was John Quincy Adams affiliated ...,[(From 1833 until was John Quincy Adams affili...,[(From 1833 until was John Quincy Adams affili...,[(For how long was John Quincy Adams affiliate...,Was John Quincy Adams affiliated to Anti-Mason...,yes,...,Until when was John Quincy Adams affiliated to...,1838.0,From 1833 until was John Quincy Adams affiliat...,1833.0,From 1833 until was John Quincy Adams affiliat...,1838.0,From when to when was John Quincy Adams affili...,"[1833, 1834, 1835, 1836, 1837, 1838]",For how long was John Quincy Adams affiliated ...,5.0
16755,[(Was Thaddeus Stevens affiliated to Anti-Maso...,[(When was Thaddeus Stevens affiliated to Anti...,[(From when to when was Thaddeus Ste

In [70]:
questions = [v1_questions, v2_questions, v3_questions, v4_questions]
return_questions = []
count = 1
for qe in questions:
    qe = qe.fillna(0)
    # qe[['until_when_an', 'left_open_an', 'right_open_an', 'when_to_when_an']] = qe[
    #     ['until_when_an', 'left_open_an', 'right_open_an', 'when_to_when_an']].astype(
    #     {"until_when_an": int, "left_open_an": int, 'right_open_an': int, 'when_to_when_an': int})
    qe = qe.loc[:, ['yes_no_qe', 'yes_no_an', 'when_qe', 'when_an', 'until_when_qe', 'until_when_an', 'left_open_qe',
                    'left_open_an', 'right_open_qe', 'right_open_an', 'when_to_when_qe', 'when_to_when_an',
                    'duration_qe', 'duration_an']]
    qe = pd.concat([qe, df.loc[:, ['object_has_years', 'subject_has_years']]], axis=1)
    qe.to_json(f'generated_data/0607/YAGO11k_questions_v{count}_0607.json')
    return_questions.append(qe)
    count += 1

In [100]:
questions = [v1_questions, v2_questions, v3_questions, v4_questions]
return_questions = []
count = 1
for qe in questions:
    qe = qe.fillna(0)
    qe = qe.loc[:, ['left_open_qe', 'left_open_an', 'right_open_qe', 'right_open_an']]
    qe = pd.concat([qe, df.loc[:, ['object_has_years', 'subject_has_years']]], axis=1)
    qe.to_json(f'generated_data/0607/YAGO11k_questions_left_right_v{count}_0607.json')
    return_questions.append(qe)
    count += 1

In [73]:
return_questions[0]

,left_open_qe,left_open_an,right_open_qe,right_open_an,object_has_years,subject_has_years
0,0,0.0,0,0.0,False,False
1,0,0.0,0,0.0,False,False
2,0,0.0,0,0.0,False,False
3,0,0.0,0,0.0,False,False
4,0,0.0,0,0.0,False,False
...,...,...,...,...,...,...
16752,From 1874 until was Benjamin Butler affiliated...,1874.0,From 1874 until was Benjamin Butler affiliated...,1889.0,False,False
16753,From 1876 until was Charles E. Cunningham affi...,1876.0,From 1876 until was Charles E. Cunningham affi...,1886.0,False,False
16754,From 1833 until was John Quincy Adams affiliat...,1833.0,From 1833 until was John Quincy Adams affiliat...,1838.0,False,False
16755,From 1828 until was Thaddeus Stevens affiliate...,1828.0,From 1828 until was Thaddeus Stevens affiliate...,1838.0,False,False


## CronKGQA

In [ ]:
df = pd.read_csv('../data/processed_datasets/TKG_readable_WIKIDATA12k.csv', index_col=0)
df

In [ ]:
# Converting until #### to 2022 indicating until today
df['until'] = df['until'].apply(lambda x: 2022 if x == '####' else x)
df

In [ ]:
unsuitable_row_ids = []
for i, entry in df.iterrows():
    if entry['from'].__contains__('#') or entry['from'].__contains__('-') or str(entry['until']).__contains__(
            '#') or str(entry['until']).__contains__('-') or entry['from'] == '':
        unsuitable_row_ids.append(i)
print(unsuitable_row_ids)

In [ ]:
df.iloc[unsuitable_row_ids, :]

In [ ]:
# Dropping all entries where the dates are ambiguous
df = df.drop(unsuitable_row_ids)

In [ ]:
start_time = time.time()

df_question = pd.DataFrame()
yes_no_dict = {'was born in': 'Was {} born in {} in {}?'}
df_question['yes_no'] = df.apply(lambda x:
                                 qm.formulate_yes_no_question(x['subject'], x['predicate'], x['object'],
                                                              x['from'], x['until'],
                                                              predicate_question_dict=yes_no_dict), axis=1)
when_dict = {'was born in': 'When was {} born in {}?'}
df_question['when'] = df.apply(lambda x:
                               qm.formulate_simple_when_question(x['subject'], x['predicate'], x['object'],
                                                                 x['from'], x['until'],
                                                                 predicate_question_dict=when_dict), axis=1)
when_to_when_dict = {'was born in': 'From when until when was {} born in {}?'}
df_question['when_to_when'] = df.apply(lambda x:
                                       qm.formulate_when_to_when_question(x['subject'], x['predicate'], x['object'],
                                                                          x['from'], x['until'],
                                                                          predicate_question_dict=when_to_when_dict,
                                                                          all_variations=False), axis=1)
from_dict = {'is married to': '{} was married to {} but when was their marriage?'}
df_question['from_when'] = df.apply(lambda x:
                                    qm.formulate_from_or_until_question(x['subject'], x['predicate'], x['object'],
                                                                        x['from'], x['until'],
                                                                        is_from_question=True,
                                                                        predicate_question_dict=when_to_when_dict,
                                                                        all_variations=False), axis=1)
until_dict = {'is married to': '{} was married to {} but when was their divorce?'}
df_question['until_when'] = df.apply(lambda x:
                                     qm.formulate_from_or_until_question(x['subject'], x['predicate'], x['object'],
                                                                         x['from'], x['until'],
                                                                         is_from_question=False,
                                                                         predicate_question_dict=when_to_when_dict,
                                                                         all_variations=False), axis=1)

left_open_dict = {'was president of': 'In the year {} {} was elected president of {} until when?'}
df_question['left_open'] = df.apply(lambda x:
                                    qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                       x['object'],
                                                                                       x['from'], x['until'],
                                                                                       right_open=False,
                                                                                       predicate_question_dict=left_open_dict),
                                    axis=1)
right_open_dict = {'was president of': 'Until the year {} {} was president of {} and when was the election?'}
df_question['right_open'] = df.apply(lambda x:
                                     qm.formulate_left_or_right_open_interval_questions(x['subject'], x['predicate'],
                                                                                        x['object'],
                                                                                        x['from'], x['until'],
                                                                                        right_open=False,
                                                                                        predicate_question_dict=right_open_dict),
                                     axis=1)

print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
convert_qm_output_to_two_df_columns(df_question, 'yes_no')
convert_qm_output_to_two_df_columns(df_question, 'when')
convert_qm_output_to_two_df_columns(df_question, 'until_when')
convert_qm_output_to_two_df_columns(df_question, 'left_open')
convert_qm_output_to_two_df_columns(df_question, 'right_open')
convert_qm_output_to_two_df_columns(df_question, 'when_to_when')

In [ ]:
df_question = df_question.fillna(0)
df_question[['until_when_an', 'left_open_an', 'right_open_an']] = df_question[
    ['until_when_an', 'left_open_an', 'right_open_an']]
.astype({"until_when_an": int, "left_open_an": int, 'right_open_an': int})

In [ ]:
df_question = df_question.loc[:,
              ['yes_no_qe', 'yes_no_an', 'when_qe', 'when_an', 'until_when_qe', 'until_when_an', 'left_open_qe',
               'left_open_an', 'right_open_qe', 'right_open_an', 'when_to_when_qe', 'when_to_when_an']]

In [ ]:
df_question.to_json('generated_data/WIKIDATA12k_questions.json')

In [ ]:
df.iloc[1, 4]

In [ ]:
pd.read_json('generated_data/WIKIDATA12k_questions.json').columns

# Adhoc creation of special questions

In [ ]:
df_question = pd.DataFrame()
yes_no_dict = {'was born in': 'Was {} born in {} in the year {}?'}
df_question['yes_no'] = df.apply(lambda x:
                                 qm.formulate_yes_no_question(x['subject'], x['predicate'], x['object'],
                                                              x['from'], x['until'],
                                                              predicate_question_dict=yes_no_dict,
                                                              clarify_question=True), axis=1)


In [ ]:
convert_qm_output_to_two_df_columns(df_question, 'yes_no')

In [ ]:
df_question.loc[:, ['yes_no_qe', 'yes_no_an']].to_json('generated_data/YAGO11k_yes_no_clarify_question.json')

In [ ]:
df_question

Yes No Robust

In [55]:
df_yes_no_robust

,qe_index,qe,an
0,0,Was Beverly Adams born in Edmonton in the year...,no
1,0,Was Beverly Adams born in Edmonton in the year...,no
2,0,Was Beverly Adams born in Edmonton in the year...,no
3,0,Was Beverly Adams born in Edmonton in the year...,no
4,0,Was Beverly Adams born in Edmonton in the year...,yes
...,...,...,...
200566,16756,Was Joseph Ritner affiliated to Anti-Masonic P...,yes
200567,16756,Was Joseph Ritner affiliated to Anti-Masonic P...,yes
200568,16756,Was Joseph Ritner affiliated to Anti-Masonic P...,yes
200569,16756,Was Joseph Ritner affiliated to Anti-Masonic P...,yes


In [ ]:
relevant_columns = ['qe']

counter = 1
df = df_yes_no_robust
# df = pd.read_json(f'{PATH}qe/0602/{file}')
for column in relevant_columns:
    question_type = column[:len(column) - 3]
    model_answer = question_type + '_model_an'
    start_time = time.time()
    for i, entry in df.iterrows():
        if entry[column] == 0:
            continue
        inputs = tokenizer.encode(entry[column], return_tensors="pt").to(device)
        outputs = model.generate(inputs)
        df.loc[i, model_answer] = tokenizer.decode(outputs[0])
    df[question_type + '_time'] = time.time() - start_time

df.to_json(f'{PATH}an/0602/answers_{version}_0602.json')